In [2]:
using Random
using Distributions
using FLoops
using Base.Threads

include("data.jl")
include("util.jl")

include("intervals/permutation.jl")
include("intervals/bootstrap.jl")
include("intervals/t.jl")

Threads.nthreads()

112

In [3]:
"""
DATA CONFIG
"""
alpha = 0.05

# data
B  = 100   # num. coverage probabilities per boxplot
S  = 4300  # num. samples per coverage probability
nx = 13    # size of group 1
ny = 5     # size of group 2
px, py = partition(nx, ny)
mc_size = try
    (binomial(nx+ny, nx) > 45_000) ? 10_000 : 0
catch
    0
end

# ALWAYS USE EXACT ENUMERATION FOR NOW
mc_size = 0

dtype = Float32
seed = 123

123

In [4]:
# POPULATION SETTINGS

Random.seed!(123)

distrTypeX = LogNormal{dtype}
X_mu = dtype.(round.(rand(B), digits=3))
X_sigma = dtype.(round.(rand(B) * 0.6, digits=3))
distrX = map(distrTypeX, X_mu, X_sigma)

distrTypeY = Gamma{dtype}
Y_shape = @. 1 / (exp(X_sigma^2) - 1)
Y_scale = @. exp(X_mu + X_sigma^2 / 2) / Y_shape
distrY = map(distrTypeY, Y_shape, Y_scale);

In [5]:
deltas = @. mean(distrX) - mean(distrY)

@show distrX[1:2]
@show distrY[1:2]
@show deltas[1:2];

distrX[1:2] = LogNormal{Float32}[LogNormal{Float32}(μ=0.906f0, σ=0.572f0), LogNormal{Float32}(μ=0.443f0, σ=0.508f0)]
distrY[1:2] = Gamma{Float32}[Gamma{Float32}(α=2.5836008f0, θ=1.1279569f0), Gamma{Float32}(α=3.3964891f0, θ=0.5216751f0)]
deltas[1:2] = Float32[0.0, 0.0]


In [6]:
Random.seed!(123)
xs = [dtype.(rand(distrX[i], nx, S)) for i in 1:B]
ys = [dtype.(rand(distrY[i], ny, S)) for i in 1:B]
@show size(ys)
ys[2]

size(ys) = (100,)


5×4300 Matrix{Float32}:
 2.47121   1.27239   1.59492  1.66024   …  2.45372   1.41319   2.43578
 0.996819  0.707802  2.58658  2.90352      3.89965   0.841454  2.32706
 1.32954   1.23017   2.34991  1.01555      0.845986  0.919507  2.23013
 2.67322   1.92672   1.46261  1.82343      0.432305  1.41197   1.50141
 0.167091  3.03278   5.45277  0.501289     0.768469  1.59987   1.65746

In [7]:
# flatten into 3D matrix
X = reshape(hcat(xs...), nx, S, B)
Y = reshape(hcat(ys...), ny, S, B)
Y[:,:,2]

5×4300 Matrix{Float32}:
 2.47121   1.27239   1.59492  1.66024   …  2.45372   1.41319   2.43578
 0.996819  0.707802  2.58658  2.90352      3.89965   0.841454  2.32706
 1.32954   1.23017   2.34991  1.01555      0.845986  0.919507  2.23013
 2.67322   1.92672   1.46261  1.82343      0.432305  1.41197   1.50141
 0.167091  3.03278   5.45277  0.501289     0.768469  1.59987   1.65746

In [8]:
function save_ci_results(results, methodId, B, S, pooled=nothing, two_sided=nothing; prefix="", dir="./")
    averages = []

    for batchId in 1:B
        batch = results[methodId, batchId, :]
        coverage = sum([j for (j, _) in batch]) / S
        width = sum([j for (_, j) in batch]) / S
        push!(averages, (coverage, width))
    end
    if isnothing(two_sided)
        save(averages, distrX[1:B], distrY[1:B], alpha, prefix=prefix, dir=dir)
    else
        alpha_ = two_sided ? alpha : alpha / 2
        save(averages, distrX[1:B], distrY[1:B], alpha_, pooled, two_sided, prefix=prefix, dir=dir)
    end
end

function save_permutation_results(results, B, S; prefix="", dir="./")
    i = 1
    per_method = []
    for two_sided in [true, false]
        for pooled in [true, false]
            save_ci_results(results, i, B, S, pooled, two_sided, prefix=prefix, dir=dir)
            i += 1
        end
    end
end

save_permutation_results (generic function with 1 method)

In [9]:
#B = 100
#S = 9605

T = Threads.nthreads()
results = Array{Union{Nothing, Tuple}, 3}(undef, 6, B, S)
fill!(results, nothing)
mc_size = 0

#@time Threads.@threads for (i,j) in collect(Iterators.product(1:B, 1:S)) # 15.52 sec on (B,S) = (5, 1800)
@time @floop ThreadedEx(basesize=div(B*S, T)) for b in 1:B, s in 1:S
    @inbounds x = X[:,s,b]
    @inbounds y = Y[:,s,b]
 
    results[1, b, s] = permInterval(x, y, deltas[b], (px, py, mc_size), true, alpha, twoSided, twoSided, 0.0005)    
    results[2, b, s] = permInterval(x, y, deltas[b], (px, py, mc_size), false, alpha, twoSided, twoSided, 0.0005)
    results[3, b, s] = permInterval(x, y, deltas[b], (px, py, mc_size), true, alpha/2, greater, smaller, 0.0005)
    results[4, b, s] = permInterval(x, y, deltas[b], (px, py, mc_size), false, alpha/2, greater, smaller, 0.0005)
    
    """
    results[5, b, s] = bootstrap(x, y, deltas[b], alpha, nsamples=10_000)
    results[6, b, s] = tconf(x, y, deltas[b], alpha, false)
    """
end

dir = "../results/" * string(nx) * "_" * string(ny) * "/2/"
save_permutation_results(results, B, S; dir=dir)
#save_ci_results(results, 6, B, S; prefix="bs", dir=dir)

2365.250478 seconds (3.88 G allocations: 28.393 TiB, 80.49% gc time, 0.47% compilation time)
dir * filename = "../results/13_5/2/twoSided_pooled_0.05.csv"
dir * filename = "../results/13_5/2/twoSided_unpooled_0.05.csv"
dir * filename = "../results/13_5/2/oneSided_pooled_0.025.csv"
dir * filename = "../results/13_5/2/oneSided_unpooled_0.025.csv"


In [ ]:
save_ci_results(results, 5, B, S; prefix="bs", dir=dir)

In [76]:
averages = []
for row in eachrow(results[5,:,:])
    coverage = sum([j for (j, _) in row]) / S
    width = sum([j for (_, j) in row]) / S
    push!(averages, (coverage, width))
end
averages

5-element Vector{Any}:
 (0.9041095890410958, 1.7169137288119696)
 (0.8964383561643836, 1.5188572278067674)
 (0.9057534246575343, 1.0491979467480967)
 (0.8997260273972603, 0.21909246020121118)
 (0.9145205479452054, 0.2478563258378473)

---

In [70]:
x = X[:,1,1]
y = Y[:,1,1]
@time permInterval(x, y, deltas[1], (px, py, 0), true, alpha, twoSided, twoSided)

permInterval(x, y, deltas[1], (px, py, 0), true, alpha/2, greater, smaller)

pval(x, y, 2.4218399175319254, (px, py, 0), true, twoSided)

  0.292847 seconds (510.31 k allocations: 55.944 MiB, 93.62% compilation time)


0.05034965f0